# API SNCF: Les retards de train

__Auteur :__ Steve Caron
__Date de creation :__ 28/01/2024

__Prerequis :__

* Python 3
* Une clef API SNCF stocké dans un environnement virtuel. Clé disponible sur : https://numerique.sncf.com/startup/api/
* Git

__Installation :__

* Cloner le projet

* Creer un environnement virtuel 

    ``py -m venv .venv``
    
* Activer l'environnement virtuel

    ``.venv/Scripts/activate``

* Installer les dépendance

    ``pip install -r requirements.txt``

__Params :__

* CLEF_API : nom sous lequel est enregistrer la cle API
* CODE_GARE : ID de la gare dont on veut récuperer les données

In [36]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import json
import datetime

In [37]:
CLEF_API = "API_KEY"
CODE_GARE = "stop_area:SNCF:87713040"

Récupération de la clé API depuis l'environnement

In [38]:
load_dotenv()
api_key = os.getenv(CLEF_API)

Je récupère la date du jour et à partir de celle-ci creer les arguments de ma requete. 

In [39]:
aujourdhui = datetime.date.today()
hier_debut_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=0, minute=0 ,second=0)
hier_fin_journee = datetime.datetime(year=aujourdhui.year, month=aujourdhui.month, day=aujourdhui.day-1, hour=23, minute=59 ,second=59)

Creation des fonctions pour coder et decoder les datetime

In [40]:
def convertir_en_string(dt):
    '''Cette fonction convertit un datetime en chaine de caractère'''
    return datetime.datetime.strftime(dt,'%Y%m%dT%H%M%S')

In [41]:
def convertir_en_datetime(str):
    '''Cette fonction convertit une chaine de caractère en datetime'''
    return datetime.datetime.strptime((str[:8]+str[9:]),'%Y%m%d%H%M%S')

In [42]:
def to_json(data,nom_fichier):
    '''Cette fonction permet d'enregistrer un fichier JSON'''
    with open(nom_fichier, "w") as fc:
        json.dump(data, fc)

In [43]:
def requete_api(gare,date):

    base_url = "https://api.sncf.com/v1/coverage/sncf"
    #Requete sans le filtre sur les trains
    requete = f"{base_url}/stop_areas/{gare}/arrivals?from_datetime={date}"
    reponse = requests.get(requete, auth=(api_key,""))
    reponse_json = reponse.json()
    to_json(reponse_json,"_test.json")
    
    return reponse_json

In [44]:
def requete_entre_dates(CODE_GARE,date_min,date_max):

    derniere_requete = date_min
    liste_toutes_arrivees = []
    liste_toutes_perturbations = []

    while derniere_requete < date_max:
        reponse = requete_api(CODE_GARE,derniere_requete)
        print(f"Requete pour la date de {derniere_requete} jusqu'a la data {date_max}")
        arrivee = reponse["arrivals"]
        perturbations = reponse["disruptions"]
        liste_toutes_arrivees += arrivee
        liste_toutes_perturbations += perturbations
        derniere_requete = arrivee[-1]["stop_date_time"]["base_departure_date_time"]
    
    print(f"{len(liste_toutes_arrivees)} arrivées récupérées")
    print(f"{len(liste_toutes_perturbations)} perturbations récupérées")
    
    return liste_toutes_arrivees,liste_toutes_perturbations


In [45]:
date_min = convertir_en_string(hier_debut_journee)
date_max = convertir_en_string(hier_fin_journee)

In [46]:
departs, perturbations = requete_entre_dates(CODE_GARE,date_min,date_max)

to_json(departs,"_departs.json")
to_json(perturbations,"_perturbations.json")

Requete pour la date de 20240202T000000 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T070730 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T073800 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T082400 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T091000 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T094000 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T103000 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T120230 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T131900 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T141900 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T155000 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T164700 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T173300 jusqu'a la data 20240202T235959
Requete pour la date de 20240202T182330 jusqu'a la data 20240202